## 深度学习报告
作业报告基于B榜最优分数编写，用中文完成（可附带英文版本），整合在Jupyter notebook文件，至少包括以下部分：
- 数据预处理
- 数据可视化
- 模型构建
- 模型训练
- 模型评估
- 陈述总结
- 参考文献，注意全部列出的参考文献需在文中引用。


---
# 1. 库


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
from torchtoolbox.transform import Cutout
import os
from PIL import Image 
import pandas as pd
from PIL import Image 
import pandas
import datetime
from torch.utils.data import random_split


from transformers import ViTFeatureExtractor, ViTForImageClassification

c:\Softwares\Anaconda\envs\Deep\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
# 2. 数据导入与处理

## 2.1 数据预处理

In [2]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    Cutout(), #遮挡增强
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[ 0.5,  0.5,  0.5])
])

## 2.2 学号信息

In [3]:
#22211360121-李凯荣-22人工智能1班
student_id = '22211360121'
#生成路径
subdir = ''

## 2.3 训练数据集

In [4]:
# 加载数据集
full_dataset = torchvision.datasets.ImageFolder(root='new data/train', transform=transform)


In [5]:
# 训练数据集划分
train_size=int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

In [6]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4,pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [7]:
print(f"训练集大小: {len(train_dataset)}, 验证集大小: {len(val_dataset)}")

训练集大小: 2844, 验证集大小: 711


In [8]:
print(f"Number of batches in train_loader: {len(train_loader)}")

Number of batches in train_loader: 89


In [9]:
print(f"Dataset size: {len(train_loader.dataset)}")

Dataset size: 2844


## 2.4 测试数据集 A

In [10]:
# 加载测试集
test_folder = 'new data/testA'
test_images = [img for img in os.listdir(test_folder) if img.endswith('.jpg')]

## 2.5 其他

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


---
# 3. 数据可视化

---
# 4. 模型构建

## 4.1 模型

In [ ]:
model = ViTForImageClassification.from_pretrained(
    'google/vit-large-patch16-224-in21k',
    num_labels=100,  # 100个鸟类类别
    ignore_mismatched_sizes=True
)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
# 冻结除最后4层外的所有层
for name, param in model.named_parameters():
    # 解冻分类头层
    if 'classifier' in name:
        param.requires_grad = True
    # 解冻最后4个Transformer块
    elif 'encoder.layer.20' in name or 'encoder.layer.21' in name or \
        'encoder.layer.22' in name or 'encoder.layer.23' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False  # 冻结其他层

# 将模型移至设备
model = model.to(device)

## 4.2 损失函数与优化器

In [14]:
# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=5e-5,  # 更小的学习率
    weight_decay=0.05,  # 更强的权重衰减
    eps=1e-8  # 数值稳定性
)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer, 
    T_max=50,  # 半周期长度
    eta_min=1e-6  # 最小学习率
)

---
# 5. 模型训练

## 5.1 设备

In [15]:
#使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

## 5. 训练

In [16]:
#早停机制
class EarlyStopping:
    def __init__(self, patience=5, verbose=True, delta=0, path='best_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path
        
    def __call__(self, val_loss, model):
        score = -val_loss
        
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter}/{self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
            
    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'验证损失改善 ({self.val_loss_min:.4f} → {val_loss:.4f}). 保存模型...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

# 初始化时指定保存路径
early_stopping = EarlyStopping(patience=5, verbose=True, path='best_model.pth')

In [20]:
from tqdm import tqdm
#轮数
num_epochs=100

# 解冻计划：每N个epoch解冻一层
unfreeze_interval = 5

# 训练模型
for epoch in range(num_epochs):  # 假设训练5个epoch
    print(f"Starting epoch {epoch+1}/{num_epochs}")
    model.train()
    train_loss=0.0
    train_correct=0
    train_total=0
    
    
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} Training")

    for inputs, labels in train_bar:
        inputs = inputs.to(device)  # 将输入数据移动到 GPU
        labels = labels.to(device)  # 将标签数据移动到 GPU

        optimizer.zero_grad() #清零梯度
        outputs = model(inputs) #前向传播
        logits=outputs.logits

        loss = criterion(logits, labels) #损失计算
        loss.backward() #反向传播
        optimizer.step() #更新参数

        train_loss+=loss.item()
        _, predicted = torch.max(logits, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()

        # 更新进度条
        train_bar.set_postfix(loss=loss.item())
        train_bar.update(1)

    train_loss = train_loss / len(train_loader)
    train_acc = 100 * train_correct / train_total
    
    # 验证阶段
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    val_bar = tqdm(val_loader, desc="Validating")
    with torch.no_grad():
        for inputs, labels in val_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            logits=outputs.logits

            loss = criterion(logits, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(logits, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            
            val_bar.set_postfix(loss=loss.item())

    val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total
    
    print(f'Epoch {epoch+1}/{num_epochs}')
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%')
    print(f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')
    
    scheduler.step()

    # 早停机制检查
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print("早停触发，停止训练")
        break

    



Starting epoch 1/100


Validating: 100%|██████████| 23/23 [00:16<00:00,  1.42it/s, loss=3.73]


Epoch 1/100
Train Loss: 3.5194, Train Acc: 63.26%
Val Loss: 3.5330, Val Acc: 55.41%
验证损失改善 (3.5914 → 3.5330). 保存模型...
Starting epoch 2/100


Validating: 100%|██████████| 23/23 [00:15<00:00,  1.46it/s, loss=3.65]


Epoch 2/100
Train Loss: 3.4548, Train Acc: 63.78%
Val Loss: 3.4522, Val Acc: 58.23%
验证损失改善 (3.5330 → 3.4522). 保存模型...
Starting epoch 3/100


...

Epoch 41/100
Train Loss: 2.4104, Train Acc: 71.24%
Val Loss: 2.4643, Val Acc: 66.39%
EarlyStopping counter: 3/5
Starting epoch 42/100


Validating: 100%|██████████| 23/23 [00:16<00:00,  1.43it/s, loss=2.37]


Epoch 42/100
Train Loss: 2.4143, Train Acc: 70.50%
Val Loss: 2.4812, Val Acc: 62.59%
EarlyStopping counter: 4/5
Starting epoch 43/100


Validating: 100%|██████████| 23/23 [00:15<00:00,  1.50it/s, loss=3.01]

Epoch 43/100
Train Loss: 2.4194, Train Acc: 70.08%
Val Loss: 2.4896, Val Acc: 64.56%
EarlyStopping counter: 5/5
早停触发，停止训练


---
# 6. 模型评估

## 6.1 模型加载

In [21]:
# 模型加载修改
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k',
    num_labels=100,
    ignore_mismatched_sizes=True
)
model.load_state_dict(torch.load('best_model.pth'))
model = model.to(device)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 6.2 推理预测

In [22]:
model=model.to(device)
model.eval()
predicts = []
idx = []

# 对测试集中的每张图像进行预测
with torch.no_grad():
    for img_name in test_images:
        img_path = os.path.join(test_folder, img_name)        
        image = Image.open(img_path).convert('RGB')  # 以RGB模式打开图像
        image = transform(image).unsqueeze(0).to(device)  # 应用预处理并增加batch维度
        outputs = model(image)
        logits=outputs.logits

        _, predicted = torch.max(logits, 1)
        predicts.append(predicted.item())
        idx.append(img_name.replace('.jpg', ''))  # 去掉文件扩展名作为ID


## 6.3 结果保存

In [23]:
# 保存预测结果到CSV文件
submission = pd.DataFrame({'id': idx, 'label': predicts})
submission['id']=submission['id'].astype(int)
submission=submission.sort_values(by='id')
submission

,id,label
0,0,56
1,1,39
237,2,33
348,3,78
459,4,52
...,...,...
137,1120,41
138,1121,54
139,1122,86
140,1123,26


In [24]:
submission.to_csv(subdir + student_id + 'submission_{}.csv'.format(
    datetime.datetime.now().strftime('%Y%m%d_%H%M%S')),
                index=False)

---
# 7. 陈述总结